In [1]:
import subprocess
import json

from utils.iter import generate_combinations

In [8]:
def gen_sbatch(config_string):
    sbatch_string = f"""#!/bin/bash

    #SBATCH --nodes=1
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=8
    #SBATCH --time=0:05:00
    #SBATCH --mem=32GB
    #SBATCH --gres=gpu:1
    #SBATCH --job-name=diffeo
    #SBATCH --mail-type=FAIL
    #SBATCH --mail-user=cm6627@nyu.edu
    #SBATCH --output=./slurm/slurm_%j.out
    #SBATCH --error=./slurm/slurm_%j.err
    
    module purge
    
    singularity exec --nv \
      --overlay /scratch/cm6627/diffeo_cnn/my_env/overlay-15GB-500K.ext3:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-val.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-test.sqf:ro \
      --overlay /vast/work/public/ml-datasets/imagenet/imagenet-train.sqf:ro \
      /scratch/work/public/singularity/cuda12.3.2-cudnn9.0.0-ubuntu-22.04.4.sif \
      /bin/bash -c "source /ext3/env.sh; python fit.py '{config_string}'" """
    return sbatch_string

def submit(config):
    config_string = str(json.dumps(config)).replace('"', '\\"')
    sbatch_string = gen_sbatch(config_string)

    with open("submit.sbatch", "w") as f:
        f.write(sbatch_string)
    
    subprocess.run(["sbatch ./submit.sbatch"], shell=True)

In [12]:
config = {
    "batch_size": [16, 32],
    "epochs": 1500,
    "model": {
        "name": "MLP_v2",
        "dropout_rate": [0.2, 0.4, 0.6],
    },
    "optimizer": {
        "name": "AdamW",
        "lr": [1e-4, 1e-5, 1e-6],
        "weight_decay": [0.02, 0.02, 2.0]
    },
    "scheduler": {
        "name": "ReduceLROnPlateau",
        "factor": 0.1,
    },
    "dataset": "resnet18_imagenet1k_train"
}

for config in generate_combinations(config):
    submit(config)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (987390148.py, line 4)

In [13]:
config = {
    "batch_size": 16,
    "epochs": 100,
    "model": {
        "name": "MLP_v2",
        "dropout_rate": 0.02,
    },
    "optimizer": {
        "name": "AdamW",
        "lr": 1e-4,
        "weight_decay": 0.002
    },
    "scheduler": {
        "name": "ReduceLROnPlateau",
        "factor": 0.1,
    },
    "dataset": "resnet18_imagenet1k_train"
}

submit(config)

Submitted batch job 57060334


In [24]:
!squeue -u cm6627

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          57060129     short ood-jupy   cm6627  R      17:09      1 cm041
          57060334     short submit.s   cm6627  R       1:05      1 cm001
